<a href="https://colab.research.google.com/github/rezu98/ULR/blob/main/UBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [306]:
cd /content

/content


In [541]:
# ubs_factor_replication.py
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.decomposition import FactorAnalysis
import matplotlib.pyplot as plt

# =========================
# 0) 사용자 설정
# =========================
INPUT_FILE = "date.xlsx"     # 입력 엑셀
DATE_COL   = "date"
VAR_COLS   = ["debt", "cons", "pir", "ptorent", "ptonation"]  # 이미 표준화된 5개 지표

# 가중치 산출 옵션
WEIGHT_FROM = "none"   # "abs" (기본) | "square"  (|λ| vs λ^2 정규화)
SHRINK_TO_EQUAL = True    # True면 동등가중(0.2씩)으로 수렴시키는 shrink 적용
SHRINK_RHO = 0.3          # 0.0(무) ~ 1.0(완전원래가중치); 0.3~0.5 권장
CAP_DEVIATION = None       # 예: 0.10 이면 각 가중치가 0.2±0.10 범위로 제한; None이면 미적용

# 점수 등급 컷오프 (UBS 컨벤션)
CUT_LOW      = 0.5
CUT_ELEVATED = 1.0
CUT_HIGH     = 1.5

OUT_XLSX = "ubs_replicate_output.xlsx"
OUT_CSV  = "ubs_replicate_scores.csv"
PLOT_PNG = "ubs_replicate_score.png"

In [542]:
import pandas as pd

# 1) 엑셀 읽기 (Sheet1)
df = pd.read_excel("date_gn.xlsx", sheet_name="Sheet1")

In [543]:
# 3) date 파싱: "YYYY-MM_DD" → "YYYY-MM-DD"로 바꿔서 날짜형으로
df["date"] = pd.to_datetime(
    df["date"].astype(str).str.replace("_", "-", regex=False),
    format="%Y-%m-%d",
    errors="raise"
)

# 4) 필요한 컬럼만(있으면) 선택하고 정렬
cols = ["date", "debt", "cons", "pir", "ptorent", "ptonation"]
df = df[[c for c in cols if c in df.columns]].sort_values("date").reset_index(drop=True)

print(df.head())

        date      debt      cons       pir   ptorent  ptonation
0 2010-03-01 -1.637965 -1.975443 -0.250783  2.022083  -0.219093
1 2010-06-01 -0.776647  1.181024 -0.333046  1.901740  -0.268800
2 2010-09-01 -0.984448 -0.332136 -0.333880  1.758708  -0.324765
3 2010-12-01 -0.212507  0.826079 -0.360930  1.300229  -0.326555
4 2011-03-01 -0.214746 -1.836071 -0.438539  1.108986  -0.439193


In [544]:

# =========================
# 2) 입력 표준화 여부 점검(선택)
# =========================
# 이미 표준화되어 있다고 했으므로 여기서는 분포를 간단히 체크만 (원하면 재표준화 가능)
# 재표준화를 원하면 아래 주석 해제:
# for c in VAR_COLS:
#     mu = df[c].mean(skipna=True); sd = df[c].std(skipna=True)
#     if sd > 0:
#         df[c] = (df[c] - mu) / sd

Z = df[VAR_COLS].copy()
mask_valid = ~Z.isna().any(axis=1)
Z_valid = Z.loc[mask_valid].values  # n_valid x 5

if Z_valid.shape[0] < 20:
    raise ValueError("유효 표본이 너무 적습니다(행>=20 권장). NaN을 확인하세요.")

In [545]:

# =========================
# 3) 요인분석(1요인)
# =========================
fa = FactorAnalysis(n_components=1, random_state=0)
fa.fit(Z_valid)

# sklearn FactorAnalysis: components_.shape = (n_components, n_features)
loadings = fa.components_[0].astype(float)  # 길이 5

# 부호 정합(위험신호 ↑ 방향): 합이 음수면 뒤집기
if loadings.sum() < 0:
    loadings = -loadings

In [546]:
loadings

# HD는 전기대비 증감, 건설투자는 수준으로 했을 때 가장 잘 나옴
# HD는 전기대비 증감, 건설투자도 전기대비 증감일 경우 나쁘지 않음

array([ 0.12335139, -0.06653703,  0.8925361 ,  0.66343587,  0.9516274 ])

In [547]:

# =========================
# 4) 적재치 → 기본 가중치
# =========================
if WEIGHT_FROM == "abs":
    base = np.abs(loadings)
elif WEIGHT_FROM == "square":
    base = loadings**2
elif WEIGHT_FROM == "none":
  base = loadings

else:
    raise ValueError("WEIGHT_FROM은 'abs' 또는 'square'만 허용합니다.")

if base.sum() == 0:
    raise ValueError("모든 적재치가 0으로 추정되었습니다. 데이터/표준화를 확인하세요.")
w0 = base / base.sum()  # 기본 가중치 (합=1)

In [548]:
# =========================
# 5) 평균 가중치로의 수렴(또는 캡) 조정
# UBS는 '도시 평균 가중치'에서의 편차를 제한한다고 밝힘.
# 단일 지역일 때는 '평균' 대용으로 '동등가중'을 사용.
equal_w = np.ones(len(VAR_COLS)) / len(VAR_COLS)

w_adj = w0.copy()

# (A) shrink toward equal weights
if SHRINK_TO_EQUAL:
    rho = float(SHRINK_RHO)
    rho = min(max(rho, 0.0), 1.0)
    w_adj = equal_w + rho * (w_adj - equal_w)

# (B) deviation cap around equal weights
if CAP_DEVIATION is not None:
    cap = float(CAP_DEVIATION)
    low = equal_w - cap
    high = equal_w + cap
    w_adj = np.minimum(np.maximum(w_adj, low), high)
    # 캡 이후 합=1로 재정규화
    s = w_adj.sum()
    if s <= 0:
        raise ValueError("캡 적용 후 가중치 합이 비정상입니다. CAP_DEVIATION을 줄이세요.")
    w_adj = w_adj / s

# 안전: 합=1 재정규화
w_adj = np.maximum(w_adj, 0.0)
w_adj = w_adj / w_adj.sum()

In [549]:
# =========================
# 6) 점수 산출 & 등급화
# =========================
# 전체 시계열에 대해 (NaN 있으면 NaN 결과)
score = (Z.values @ w_adj)
df_out = df[[DATE_COL]].copy()
df_out["score"] = score

def classify(val):
    if pd.isna(val):
        return np.nan
    if val > CUT_HIGH:
        return "High"
    elif val > CUT_ELEVATED:
        return "Elevated"
    elif val > CUT_LOW:
        return "Moderate"
    else:
        return "Low"

df_out["grade"] = df_out["score"].apply(classify)

In [550]:

# =========================
# 7) 결과 표/그림/파일 저장
# =========================
weights_df = pd.DataFrame({
    "variable": VAR_COLS,
    "loading": loadings,
    "weight_raw": w0,
    "weight_final": w_adj
})

# 저장
with pd.ExcelWriter(OUT_XLSX) as writer:
    weights_df.to_excel(writer, sheet_name="weights", index=False)
    df_out.to_excel(writer, sheet_name="scores", index=False)

df_out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

# 간단 플롯
plt.figure(figsize=(10,4))
plt.plot(df_out[DATE_COL], df_out["score"], lw=1.2)
plt.axhline(CUT_LOW,      ls="--")
plt.axhline(CUT_ELEVATED, ls="--")
plt.axhline(CUT_HIGH,     ls="--")
plt.title("UBS-style Bubble Risk Score (Single City)")
plt.xlabel("Date")
plt.ylabel("Score")
plt.tight_layout()
plt.savefig(PLOT_PNG, dpi=150)
plt.close()

# 콘솔 요약 출력
print("=== Factor Loadings & Weights ===")
print(weights_df.round(4))
print()
print("Latest score/grade:",
      df_out.dropna(subset=["score"]).iloc[-1][["score","grade"]].to_dict())
print(f"\nSaved files: {OUT_XLSX}, {OUT_CSV}, {PLOT_PNG}")

=== Factor Loadings & Weights ===
    variable  loading  weight_raw  weight_final
0       debt   0.1234      0.0481        0.1544
1       cons  -0.0665     -0.0259        0.1322
2        pir   0.8925      0.3480        0.2444
3    ptorent   0.6634      0.2587        0.2176
4  ptonation   0.9516      0.3711        0.2513

Latest score/grade: {'score': 1.758389206539659, 'grade': 'High'}

Saved files: ubs_replicate_output.xlsx, ubs_replicate_scores.csv, ubs_replicate_score.png
